In [1]:
import json
import os
import glob
from tqdm import tqdm

import numpy as np
import pandas as pd

from utils.keypoints import rescale_keypoints, read_keypoints

import torch
from model.models import StopPoseNet, SequenceRecognitionNet

import cv2

In [2]:
STOP_POSE_NET_PATH = './model/stop_pose_detector.pt'
# JSON_FILES_PATH = ['./data/stop_pose_labelsSep-13-2020_2021.json']
JSON_KEYPOINTS_BASE_PATH = './data/keypoints/internal_resource_2337_fps10'
IMAGES_BASE_PATH = './data/video_frames/internal_resource_2337_fps10'
RANDOM_STATE = 42

DEVICE = torch.device('cpu')

In [3]:
state_dict = torch.load(STOP_POSE_NET_PATH)
stop_pose_net = StopPoseNet().to(DEVICE)
stop_pose_net.load_state_dict(state_dict)
stop_pose_net.eval()

StopPoseNet(
  (fc1): Linear(in_features=114, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=128, bias=True)
  (fc4): Linear(in_features=128, out_features=32, bias=True)
  (fc5): Linear(in_features=32, out_features=2, bias=True)
  (dropout1): Dropout(p=0.1, inplace=False)
  (relu): ReLU()
)

In [4]:
# pd.set_option('display.max_colwidth', 200)
# labels_json = None
# for json_file in JSON_FILES_PATH:
#     with open(json_file, 'r') as f:
#         for row in f.readlines():
#             labels_json = json.loads(row)

# df = pd.DataFrame(labels_json)

# def get_keypoints_path(file_path):
#     file_basename = os.path.basename(file_path).split('.')[0]
#     keypoints_path = os.path.realpath(os.path.join(JSON_KEYPOINTS_BASE_PATH, f'{file_basename}_keypoints.json'))
    
#     return keypoints_path

# df['keypoints_path'] = df['file_path'].apply(get_keypoints_path)
# df

In [5]:
pd.set_option('display.max_colwidth', 200)

# Get image file path
def get_image_path(json_keypoints_path):
    file_basename = os.path.basename(json_keypoints_path).split('.')[0]
    file_basename = file_basename.replace('_keypoints', '')
    image_path = os.path.join(IMAGES_BASE_PATH, f'{file_basename}.jpg')
    
    return image_path

# Load JSON with keypoints
json_files = []
for file_path in glob.glob(f'{JSON_KEYPOINTS_BASE_PATH}/*.json'):
    json_files.append({
        'keypoints_path': os.path.realpath(file_path),
        'file_path': os.path.realpath(get_image_path(file_path))
        
    })
print(json_files)
    
df = pd.DataFrame(data=json_files).sort_values(by=['keypoints_path']).reset_index().drop(columns=['index'])

df.head()

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



,keypoints_path,file_path
0,/home/tom/Desktop/projects/sign-language/data/keypoints/internal_resource_2337_fps10/internal_resource_2337_000100_keypoints.json,/home/tom/Desktop/projects/sign-language/data/video_frames/internal_resource_2337_fps10/internal_resource_2337_000100.jpg
1,/home/tom/Desktop/projects/sign-language/data/keypoints/internal_resource_2337_fps10/internal_resource_2337_000101_keypoints.json,/home/tom/Desktop/projects/sign-language/data/video_frames/internal_resource_2337_fps10/internal_resource_2337_000101.jpg
2,/home/tom/Desktop/projects/sign-language/data/keypoints/internal_resource_2337_fps10/internal_resource_2337_000102_keypoints.json,/home/tom/Desktop/projects/sign-language/data/video_frames/internal_resource_2337_fps10/internal_resource_2337_000102.jpg
3,/home/tom/Desktop/projects/sign-language/data/keypoints/internal_resource_2337_fps10/internal_resource_2337_000103_keypoints.json,/home/tom/Desktop/projects/sign-language/data/video_frames/internal_resource_2337_fps10/internal_resource_2337_000103.jpg
4,/home/tom/Desktop/projects/sign-language/data/keypoints/internal_resource_2337_fps10/internal_resource_2337_000104_keypoints.json,/home/tom/Desktop/projects/sign-language/data/video_frames/internal_resource_2337_fps10/internal_resource_2337_000104.jpg


In [6]:
stop_pose_net.eval()
stop_pose_preds = []

for row in df.itertuples():
    keypoints = read_keypoints(row.keypoints_path)
    if not keypoints.size:
        continue
        
    keypoints = rescale_keypoints(keypoints)
    keypoints_tensor = torch.FloatTensor(keypoints).to(DEVICE).unsqueeze(0)
    
    pred = stop_pose_net(keypoints_tensor).detach().cpu().numpy()
    pred = np.argmax(pred)
    stop_pose_preds.append(pred)

In [7]:
def extract_sequences(vector):
    # Create an array that is 1 where a is 0, and pad each end with an extra 0.
    is_one = np.concatenate(([0], np.equal(np.array(vector), 0).view(np.int8), [0]))
    diff_abs = np.abs(np.diff(is_one))
    
    # Runs start and end where absdiff is 1.
    ranges = np.where(diff_abs == 1)[0].reshape(-1, 2)
    return ranges

sequences = extract_sequences(stop_pose_preds)

In [8]:
sequences.shape

(317, 2)

In [9]:
video_output_dir = './data/rendered_video/internal_resource_2337_fps10'
fps = 10
sequence_info = []

for i, sequence_range in tqdm(enumerate(sequences)):
    
    video_name = f'internal_resource_{sequence_range[0]}_{sequence_range[1]}.avi'
    video_path = os.path.join(video_output_dir, video_name)
    
    images = df.iloc[sequence_range[0]: sequence_range[1]]['file_path'].to_numpy().tolist()
    keypoints = df.iloc[sequence_range[0]: sequence_range[1]]['keypoints_path'].to_numpy().tolist()
    frame = cv2.imread(images[0])
    height, width, layers = frame.shape
    
    video = cv2.VideoWriter(video_path, 0, fps, (width,height))

    for image_path in images:
        video.write(cv2.imread(image_path))

    video.release()
    
    # Save info to df
    sequence_info.append(
        {
            'id': i,
            'video_path': video_path,
            'image_path': images,
            'keypoints_path': keypoints
        }
    )
print('Done')

317it [04:03,  1.30it/s]

Done


In [10]:
labels_output = []

for sequence in sequence_info:
    #clear_output(wait=True)
    print(sequence['video_path'])
    
    label = input('Provide label:')
       
    labels_output.append({
        'id': sequence['id'],
        'video_path': sequence['video_path'],
        'label': label})


./data/rendered_video/internal_resource_2337_fps10/internal_resource_13_34.avi


Provide label: Hello


./data/rendered_video/internal_resource_2337_fps10/internal_resource_46_65.avi


Provide label: Hello


./data/rendered_video/internal_resource_2337_fps10/internal_resource_77_101.avi


...

Provide label: Bye bye


./data/rendered_video/internal_resource_2337_fps10/internal_resource_11973_11996.avi


Provide label: Bye bye


./data/rendered_video/internal_resource_2337_fps10/internal_resource_12003_12019.avi


Provide label: ignore


In [14]:
sequence_labels = []
for seq, labels in zip(sequence_info, labels_output):
    sequence_labels.append({**seq, **labels})
    
sequence_labels[0]

{'id': 0,
 'video_path': './data/rendered_video/internal_resource_2337_fps10/internal_resource_13_34.avi',
 'image_path': ['/home/tom/Desktop/projects/sign-language/data/video_frames/internal_resource_2337_fps10/internal_resource_2337_000113.jpg',
  '/home/tom/Desktop/projects/sign-language/data/video_frames/internal_resource_2337_fps10/internal_resource_2337_000114.jpg',
  '/home/tom/Desktop/projects/sign-language/data/video_frames/internal_resource_2337_fps10/internal_resource_2337_000115.jpg',
  '/home/tom/Desktop/projects/sign-language/data/video_frames/internal_resource_2337_fps10/internal_resource_2337_000116.jpg',
  '/home/tom/Desktop/projects/sign-language/data/video_frames/internal_resource_2337_fps10/internal_resource_2337_000117.jpg',
  '/home/tom/Desktop/projects/sign-language/data/video_frames/internal_resource_2337_fps10/internal_resource_2337_000118.jpg',
  '/home/tom/Desktop/projects/sign-language/data/video_frames/internal_resource_2337_fps10/internal_resource_2337_000

In [15]:
# Prepare template for labeling

import time

t = time.localtime()
timestamp = time.strftime('%b-%d-%Y_%H%M', t)

output_file_path = os.path.join('./data/labels/sequence', f'sequence_labels{timestamp}.json')
with open(output_file_path, 'w') as outfile:
    json.dump(sequence_labels, outfile)

In [16]:
df_all = pd.DataFrame(sequence_labels)
df_all.label.value_counts()

You               60
My                50
See you around    42
Name              35
Meet              30
Nice              28
Bye bye           21
Tom               19
ignore            13
Good day          10
Hello              7
Bye                2
Name: label, dtype: int64